In [1]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existan varios agentes por celda, elegimos ''MultiGrid''.
from mesa.space import MultiGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd
import random

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [2]:
""" Construcción de la habitación, es el estado de los diferentes agentes
    Se pide el modelo de donde se obtendrá la información
    Devuelve la matriz con la información de los estados de los agentes"""
def get_room(model):
    room = np.zeros( (model.grid.width, model.grid.height) )
    for (content, x, y) in model.grid.coord_iter():
        for data_content in content:
            if isinstance(data_content, Robot):
                room[x][y] = 2
            else:
                room[x][y] = data_content.state
    return room

""" Agente de cada celda o parte del piso, define si el estado de limpio o sucio"""
class Cell(Agent):
    
    CLEAN = 0
    DIRTY = 1
    
    def __init__(self, pos, model, state = CLEAN):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.state = state

""" Agente del robot, establece su movimiento actual y próximo.
    En caso de que la celda esté sucia la limpiará, de lo contrario
    se moverá a una de sus 8 celdas vecinas de forma aleatoria"""
class Robot(Agent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.newPosition = None
        self.moves = 0
    
    def step(self):
        neighbors = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = True)
        
        for neighbour in neighbors:
            if neighbour.pos == self.pos:
                if neighbour.state == neighbour.DIRTY:
                    neighbour.state = neighbour.CLEAN
                    self.newPosition = self.pos
                else:
                    neighborhood = self.model.grid.get_neighborhood(
                        self.pos,
                        moore = True,
                        include_center = False)
                    newPosition = self.random.choice(neighborhood)
                    self.newPosition = newPosition
                break
        
        """Conteo de movimientos"""
        if self.pos != self.newPosition:
            self.moves += 1
        
        self.model.grid.move_agent(self, self.newPosition)

""" Modelo de la habitación, establece parámetros de sus agentes y el entorno."""
class Room(Model):
    
    def __init__(self, m, n, agentNumber, dirtyCellPercentage):
        self.agentNumber = agentNumber
        self.dirtyCellPercentage = dirtyCellPercentage
        self.cleanPercentage = 1 - dirtyCellPercentage
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
        
        """Llena de manera aleatoria las celdas sucias necesarias en la habitación"""
        dirty_cells = int((m * n) * dirtyCellPercentage)
        emptyCellsList = list(self.grid.empties)
        
        for cells in range(dirty_cells):
            empty_cell = random.choice(emptyCellsList)
            cell = Cell(empty_cell, self)
            cell.state = cell.DIRTY
            self.grid.place_agent(cell, empty_cell)
            self.schedule.add(cell)
            emptyCellsList.remove(empty_cell)
        
        """El resto de las celdas se toman como limpias"""
        emptyCellsList = list(self.grid.empties)
        for cells in emptyCellsList:
            cell = Cell(cells, self)
            self.grid.place_agent(cell, cells)
            self.schedule.add(cell)
        
        """Los robots se posicionan en la celda [1,1]"""
        for i in range(agentNumber):
            robot = Robot(i, self)
            self.grid.place_agent(robot, (1,1))
            self.schedule.add(robot)
        
        """Guardado de datos"""
        self.data_collector = DataCollector(
            model_reporters={'Room': get_room},
            agent_reporters={'Movements': lambda a: getattr(a, 'moves', None)})
        
    def step(self):
        self.data_collector.collect(self)
        self.schedule.step()    
    
    """Se calcula el porcentaje de limpieza"""
    def cleanAllCells(self):
        cleanCells = 0
        for cell in self.grid.coord_iter():
            cellContent, x, y = cell
            for content in cellContent:
                if isinstance(content, Cell) and content.state == content.CLEAN:
                    cleanCells += 1
        
        self.cleanPercentage = cleanCells / (self.grid.width * self.grid.height)
        if self.cleanPercentage == 1:
            return True
        else:
            return False
        

In [47]:
# Dimensión de la habitación
M = 10
N = 70

# Numero de agentes
Agent_Number = 50

# Porcentaje de celdas sucias en decimal
Dirty_Cells_Percentage = 0.4

# Tiempo máximo de ejecución (segundos)
Max_Execution_Time = 3

# Inicialización del tiempo
start_timer = time.time()

# Construcción del modelo
model = Room(M, N, Agent_Number, Dirty_Cells_Percentage)

# Ejecución de la simulación
while((time.time() - start_timer) < Max_Execution_Time and not model.cleanAllCells()):
    model.step()

# TIempo de ejecución de la simulación
Execution_Time = str(datetime.timedelta(seconds=(time.time() - start_timer)))

In [48]:
# Se obtiene la información del colector de datos para su visualización
rooms = model.data_collector.get_model_vars_dataframe()

In [49]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(rooms.iloc[0][0], cmap='Pastel1')

def animate(i):
    patch.set_data(rooms.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(rooms))

In [50]:
print('Dimensión de la habitación: ', M, ' x ', N)
print('Número de robots: ', Agent_Number)
print('Porcentaje de celdas sucias: ', Dirty_Cells_Percentage*100, '%')
print('Tiempo máximo de ejecución: ', Max_Execution_Time, 's')
print('Tiempo necesario de limpieza:', Execution_Time)
print('Porcentaje de celdas limpias al concluir:', model.cleanPercentage*100, '%')

moves = model.data_collector.get_agent_vars_dataframe()
print('Número de movimientos relizados por los agentes:', moves.tail()['Movements'].sum())
anim

Dimensión de la habitación:  10  x  70
Número de robots:  50
Porcentaje de celdas sucias:  40.0 %
Tiempo máximo de ejecución:  3 s
Tiempo necesario de limpieza: 0:00:01.434889
Porcentaje de celdas limpias al concluir: 100.0 %
Número de movimientos relizados por los agentes: 2464.0
